In [1]:
class_names = [
    "Apple___Apple_scab",
    "Apple___Black_rot",
    "Apple___Cedar_apple_rust",
    "Apple___healthy",
    "Blueberry___healthy",
    "Cherry_(including_sour)___Powdery_mildew",
    "Cherry_(including_sour)___healthy",
    "Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot",
    "Corn_(maize)___Common_rust_",
    "Corn_(maize)___Northern_Leaf_Blight",
    "Corn_(maize)___healthy",
    "Grape___Black_rot",
    "Grape___Esca_(Black_Measles)",
    "Grape___Leaf_blight_(Isariopsis_Leaf_Spot)",
    "Grape___healthy",
    "Orange___Haunglongbing_(Citrus_greening)",
    "Peach___Bacterial_spot",
    "Peach___healthy",
    "Pepper,_bell___Bacterial_spot",
    "Pepper,_bell___healthy",
    "Potato___Early_blight",
    "Potato___Late_blight",
    "Potato___healthy",
    "Raspberry___healthy",
    "Soybean___healthy",
    "Squash___Powdery_mildew",
    "Strawberry___Leaf_scorch",
    "Strawberry___healthy",
    "Tomato___Bacterial_spot",
    "Tomato___Early_blight",
    "Tomato___Late_blight",
    "Tomato___Leaf_Mold",
    "Tomato___Septoria_leaf_spot",
    "Tomato___Spider_mites Two-spotted_spider_mite",
    "Tomato___Target_Spot",
    "Tomato___Tomato_Yellow_Leaf_Curl_Virus",
    "Tomato___Tomato_mosaic_virus",
    "Tomato___healthy"
]

# Print the class names
for name in class_names:
    
    print(name)


Apple___Apple_scab
Apple___Black_rot
Apple___Cedar_apple_rust
Apple___healthy
Blueberry___healthy
Cherry_(including_sour)___Powdery_mildew
Cherry_(including_sour)___healthy
Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot
Corn_(maize)___Common_rust_
Corn_(maize)___Northern_Leaf_Blight
Corn_(maize)___healthy
Grape___Black_rot
Grape___Esca_(Black_Measles)
Grape___Leaf_blight_(Isariopsis_Leaf_Spot)
Grape___healthy
Orange___Haunglongbing_(Citrus_greening)
Peach___Bacterial_spot
Peach___healthy
Pepper,_bell___Bacterial_spot
Pepper,_bell___healthy
Potato___Early_blight
Potato___Late_blight
Potato___healthy
Raspberry___healthy
Soybean___healthy
Squash___Powdery_mildew
Strawberry___Leaf_scorch
Strawberry___healthy
Tomato___Bacterial_spot
Tomato___Early_blight
Tomato___Late_blight
Tomato___Leaf_Mold
Tomato___Septoria_leaf_spot
Tomato___Spider_mites Two-spotted_spider_mite
Tomato___Target_Spot
Tomato___Tomato_Yellow_Leaf_Curl_Virus
Tomato___Tomato_mosaic_virus
Tomato___healthy


In [2]:
import torch
import torch.nn as nn
from torchsummary import summary

VGG_types = {
    "VGG11": [64, "M", 128, "M", 256, 256, "M", 512, 512, "M", 512, 512, "M"],
    "VGG13": [64, 64, "M", 128, 128, "M", 256, 256, "M", 512, 512, "M", 512, 512, "M"],
    "VGG16": [
        64,
        64,
        "M",
        128,
        128,
        "M",
        256,
        256,
        256,
        "M",
        512,
        512,
        512,
        "M",
        512,
        512,
        512,
        "M",
    ],
    "VGG19": [
        64,
        64,
        "M",
        128,
        128,
        "M",
        256,
        256,
        256,
        256,
        "M",
        512,
        512,
        512,
        512,
        "M",
        512,
        512,
        512,
        512,
        "M",
    ],
}

class VGGnet(nn.Module):
    def __init__(self, in_channels=3, num_classes=38):
        super(VGGnet, self).__init__()
        self.in_channels = in_channels
        self.conv_layers = self.create_conv_layers(VGG_types["VGG19"])
        self.global_avg_pooling = nn.AdaptiveAvgPool2d(output_size=(1, 1))

        # Modify the fully connected layers to match the new input size
        self.fcs = nn.Sequential(
            nn.Linear(512, 1024),  # Adjust input size from 512*7*7 to 512
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(1024, 1024),  # Example: Adding an extra layer
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(1024, num_classes),
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.global_avg_pooling(x)
        x = x.view(x.size(0), -1)
        x = self.fcs(x)
        return x

    def create_conv_layers(self, architecture):
        layers = []
        in_channels = self.in_channels

        for x in architecture:
            if type(x) == int:
                out_channels = x

                layers += [
                    nn.Conv2d(
                        in_channels=in_channels,
                        out_channels=out_channels,
                        kernel_size=(3, 3),
                        stride=(1, 1),
                        padding=(1, 1),
                    ),
                    nn.BatchNorm2d(x),
                    nn.ReLU(),
                ]
                in_channels = x
            elif x == "M":
                layers += [nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))]

        return nn.Sequential(*layers)

device = "cuda" if torch.cuda.is_available() else "cpu"
model = VGGnet(in_channels=3, num_classes=38).to(device)


freeze_layers = 5
for i, param in enumerate(model.parameters()):
    if i < freeze_layers:
        param.requires_grad = False

# Verify which layers are frozen
for name, param in model.named_parameters():
    if not param.requires_grad:
        print(f'Layer {name} is frozen.')

# Print model summary
summary(model.to(device), (3, 224, 224))  # Adjust input size as needed


Layer conv_layers.0.weight is frozen.
Layer conv_layers.0.bias is frozen.
Layer conv_layers.1.weight is frozen.
Layer conv_layers.1.bias is frozen.
Layer conv_layers.3.weight is frozen.
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
       BatchNorm2d-2         [-1, 64, 224, 224]             128
              ReLU-3         [-1, 64, 224, 224]               0
            Conv2d-4         [-1, 64, 224, 224]          36,928
       BatchNorm2d-5         [-1, 64, 224, 224]             128
              ReLU-6         [-1, 64, 224, 224]               0
         MaxPool2d-7         [-1, 64, 112, 112]               0
            Conv2d-8        [-1, 128, 112, 112]          73,856
       BatchNorm2d-9        [-1, 128, 112, 112]             256
             ReLU-10        [-1, 128, 112, 112]               0
           Conv2d-11        [-1, 128, 112, 11

In [ ]:
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import torch
import torch.nn.functional as F
import torchvision.transforms as transforms
import numpy as np

# Define the path to the single test image you want to test
single_test_image_path = 'test/test/'

# Define the transformation for the single test image
test_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Create the main window
root = tk.Tk()
root.title("Plant leaf disease detection")
tk.Label(root,text="PLANT LEAF DISEASE DETECTION USING VGG19",width=10,height=4,bg="lightblue",font='arial 20 bold').pack(side=tk.TOP,fill=tk.X)
display_width = 600
display_height = 600

default_image_path = 'Sample-Images-from-Plant-village-dataset.png'
default_image = Image.open(default_image_path)
default_image = default_image.resize((display_width, display_height))  # Resize the image to fit the window

# Convert the image to PhotoImage
image_on_label = ImageTk.PhotoImage(default_image)
# Create a label to display the image
image_label = tk.Label(root, image=image_on_label)
image_label.pack(side=tk.LEFT,padx=10,pady=10)


# Load the trained model
model = VGGnet(in_channels=3, num_classes=38)
model.load_state_dict(torch.load('best_model_epoch_5.pt', map_location=device))
model.eval()  # Set the model to evaluation mode

def upload_image():
    global single_test_image_path
    single_test_image_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg *.jpeg *.png")])
    update_display()

def update_display():
    # Load the single test image and apply the transformation
    single_test_image = Image.open(single_test_image_path).convert('RGB')
    input_image = test_transform(single_test_image).unsqueeze(0)  # Add a batch dimension

    # Perform prediction
    with torch.no_grad():
        output = model(input_image)

    # Get predicted class and confidence
    predicted_class = torch.argmax(output, dim=1).item()
    confidence = F.softmax(output, dim=1)[0][predicted_class].item()
    predicted_class_name = class_names[predicted_class]

    # Convert the input image to NumPy array
    input_image_np = input_image.squeeze(0).permute(1, 2, 0).numpy()
    input_image_np = (input_image_np * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])).clip(0, 1)  # Reverse normalization

    # Update the display
    predicted_image = ImageTk.PhotoImage(Image.fromarray((input_image_np * 255).astype('uint8')))
    predicted_label.config(image=predicted_image)
    predicted_label.image = predicted_image

    result_label.config(text=f'Predicted: {predicted_class_name} ({confidence:.2f})')

text_above_button = tk.Label(root, text="Upload a image to predict the disease", font=("Helvetica", 16))
text_above_button.pack(pady=10)

# Create and place widgets
upload_button = tk.Button(root, text="Upload Image", command=upload_image)
upload_button.pack(pady=10)

predicted_label = tk.Label(root)
predicted_label.pack()

result_label = tk.Label(root, text="", font=("Helvetica", 16))
result_label.pack(pady=10)

# Start the Tkinter main loop
root.mainloop()
